In [1]:
"""
Se pretende realizar un modelo que pueda predecir posibles inundaciones a través de imágenes de un mismo río en diferentes estados para intentar prevenir
sucesos como el de la DANA de Valencia del 29/10/2024 y que el propio modelo sea capaz de avisar a la población en caso de una posible riada ya que preferimos un Falso Positivo antes
de un Falso Negativo.

Los significados para cada letra son los siguientes: A -> Alto nivel del agua, B -> Nivel de agua creciendo gradualmente, C -> Nivel normal de agua.
"""

'\nSe pretende realizar un modelo que pueda predecir posibles inundaciones a través de imágenes de un mismo río en diferentes estados para intentar prevenir\nsucesos como el de la DANA de Valencia del 29/10/2024 y que el propio modelo sea capaz de avisar a la población en caso de una posible riada ya que preferimos un Falso Positivo antes de un Falso Negativo\n'

In [2]:
# Lo primero que vamos a hacer es montar el Drive ya que tenemos todas las imágenes subidas en nuestra carpeta de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# A continuación vamos a importar las librerias que vamos a utilizar a lo largo del proyecto
!pip install tensorflow keras
import tensorflow as tf
print("Versión de TensorFlow:", tf.__version__)
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from google.colab.patches import cv2_imshow

Versión de TensorFlow: 2.18.0


In [11]:
# A continuación importamos los datos de train y test

train_data_dir = ("/content/drive/MyDrive/RAIN_DATA/train")
test_data_dir = ("/content/drive/MyDrive/RAIN_DATA/test")

In [12]:
"""
Si ejecutamos las siguientes celdas de código va a saltar un mensaje indicando que hay 0 Imágenes ya que keras no es capaz de encontrar las imágenes porque están todas en la misma carpeta,
por eso voy a ejecutar este script y que las imágenes queden ordenadas en 3 carpetas llamadas A, B y C almacenando cada imágen en su carpeta correspondiente.
"""

import shutil

# Ruta a los directorios
train_dir = "/content/drive/MyDrive/RAIN_DATA/train"
test_dir = "/content/drive/MyDrive/RAIN_DATA/test"

# Crear subcarpetas si no existen
for folder in ['A', 'B', 'C']:
    os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir, folder), exist_ok=True)

# Función para mover imágenes a su subcarpeta
def organizar_imagenes(base_dir):
    for file in os.listdir(base_dir):
        file_path = os.path.join(base_dir, file)

        # Asegurarse de que sea un archivo y no una carpeta
        if os.path.isfile(file_path):
            if file.startswith('A'):
                shutil.move(file_path, os.path.join(base_dir, 'A', file))
            elif file.startswith('B'):
                shutil.move(file_path, os.path.join(base_dir, 'B', file))
            elif file.startswith('C'):
                shutil.move(file_path, os.path.join(base_dir, 'C', file))

# Organizar imágenes en train y test
organizar_imagenes(train_dir)
organizar_imagenes(test_dir)

print("Organización completada correctamente.")

Organización completada correctamente.


In [13]:
# Antes de entrenar nuestro modelo con estos datos vamos a realizar Data Augmentation para tener más cantidad de imágenes con las que entrenar, así es más probable obtener mejores resultados

train_datagen = ImageDataGenerator(
    rescale=1.0/255,         # Normalización de píxeles (0 a 1)
    rotation_range=30,       # Rotación aleatoria hasta 30 grados
    width_shift_range=0.2,   # Desplazamiento horizontal hasta 20%
    height_shift_range=0.2,  # Desplazamiento vertical hasta 20%
    shear_range=0.2,         # Inclinación de la imagen
    zoom_range=0.2,          # Zoom aleatorio del 20%
    horizontal_flip=True,    # Inversión horizontal
    fill_mode='nearest'      # Relleno de píxeles vacíos
)

# Para test no vamos a aplicar DataAugmentation ya que lo único que queremos es comprobar si el modelo puede predecir correctamente
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [14]:
# A continuación vamos a cargar las imágenes tanto de train como de test
train_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=("224", "224"),  # Tamaño de las imágenes
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=("224", "224"),
    batch_size=32,
    class_mode='categorical'
)

Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.


In [ ]:
# Una vez ya tenemos las imágenes identificadas vamos a crear el modelo con el que vamos a trabajar
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Reduce overfitting
    Dense(3, activation='softmax')  # 3 clases: A, B, C
])

# Compilamos el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen de la arquitectura
model.summary()
